In [1]:
pip install celltypist==1.6.1

  Obtaining dependency information for celltypist==1.6.1 from https://files.pythonhosted.org/packages/8f/d8/f60e2df761a08bff1260a83dc9543a2f72364d7cda909eab174d69e216be/celltypist-1.6.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 98.1 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scanpy

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install numpy==1.24.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 86.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
seaborn 0.12.2 requires numpy!=1.24.0,>=1.17, but you have numpy 1.24.0 which is incompatible.
ydata-profiling 4.4.0 requires numpy<1.24,>=1.16.0, but you have numpy 1.24.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import celltypist
from celltypist import models
import scanpy as sc
import pandas as pd 
import numpy as np
import anndata
import re
import h5py
import scipy.sparse as scs
import concurrent.futures

In [2]:
def read_mat(h5_con):
    mat = scs.csc_matrix(
        (h5_con['matrix']['data'][:], # Count values
         h5_con['matrix']['indices'][:], # Row indices
         h5_con['matrix']['indptr'][:]), # Pointers for column positions
        shape = tuple(h5_con['matrix']['shape'][:]) # Matrix dimensions
    )
    return mat

# define a function to obeservation (i.e. metadata)

def read_obs(h5con):
    bc = h5con['matrix']['barcodes'][:]
    bc = [x.decode('UTF-8') for x in bc]

    # Initialized the DataFrame with cell barcodes
    obs_df = pd.DataFrame({ 'barcodes' : bc })

    # Get the list of available metadata columns
    obs_columns = h5con['matrix']['observations'].keys()

    # For each column
    for col in obs_columns:
        # Read the values
        values = h5con['matrix']['observations'][col][:]
        # Check for byte storage
        if(isinstance(values[0], (bytes, bytearray))):
            # Decode byte strings
            values = [x.decode('UTF-8') for x in values]
        # Add column to the DataFrame
        obs_df[col] = values
    
    return obs_df
# define a function to construct anndata object from a h5 file
def read_h5_anndata(h5_file):
    h5_con = h5py.File(h5_file, mode = 'r')
    # extract the expression matrix
    mat = read_mat(h5_con)
    # extract gene names
    genes = h5_con['matrix']['features']['name'][:]
    genes = [x.decode('UTF-8') for x in genes]
    # extract metadata
    obs_df = read_obs(h5_con)
    # construct anndata
    adata = anndata.AnnData(mat.T,
                             obs = obs_df)
    # make sure the gene names aligned
    adata.var_names = genes

    adata.var_names_make_unique()
    return adata

def get_last_pattern(inputstr):
    pattern = r"[^/]+(?=$)"
    match = re.search(pattern, inputstr)
    if match:
        return match.group(0)
    else:
        return ""

In [3]:
meta_data=pd.read_csv("hise_meta_data_2023-11-19.csv")

In [4]:
models.download_models(force_update = True)

📜 Retrieving model list from server https://celltypist.cog.sanger.ac.uk/models/models.json
📚 Total models in list: 44
📂 Storing models in /root/.celltypist/data/models
💾 Downloading model [1/44]: Immune_All_Low.pkl
💾 Downloading model [2/44]: Immune_All_High.pkl
💾 Downloading model [3/44]: Adult_CynomolgusMacaque_Hippocampus.pkl
💾 Downloading model [4/44]: Adult_Human_PancreaticIslet.pkl
💾 Downloading model [5/44]: Adult_Human_Skin.pkl
💾 Downloading model [6/44]: Adult_Mouse_Gut.pkl
💾 Downloading model [7/44]: Adult_Mouse_OlfactoryBulb.pkl
💾 Downloading model [8/44]: Adult_Pig_Hippocampus.pkl
💾 Downloading model [9/44]: Adult_RhesusMacaque_Hippocampus.pkl
💾 Downloading model [10/44]: Autopsy_COVID19_Lung.pkl
💾 Downloading model [11/44]: COVID19_HumanChallenge_Blood.pkl
💾 Downloading model [12/44]: COVID19_Immune_Landscape.pkl
💾 Downloading model [13/44]: Cells_Fetal_Lung.pkl
💾 Downloading model [14/44]: Cells_Intestinal_Tract.pkl
💾 Downloading model [15/44]: Cells_Lung_Airway.pkl
💾 Dow

In [ ]:
def process_data(meta_data_sub):
    # Process the subset of meta_data
    results = []
    for file_name in meta_data_sub:
        result = read_h5_anndata(file_name)
        results.append(result)
    adata = anndata.concat(results)
    del results
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata.obs.index = adata.obs['barcodes']
    predictions = celltypist.annotate(adata, model='Immune_All_Low.pkl', majority_voting=True)
    for i in adata.obs['pbmc_sample_id'].unique():
        barcodes=adata.obs[adata.obs['pbmc_sample_id'] == i].index.tolist()
        predictions.predicted_labels.loc[barcodes,:].to_csv("Labels_Celltypist/" + i + "_Low.csv")
    
    predictions = celltypist.annotate(adata, model='Immune_All_High.pkl', majority_voting=True)
    for i in adata.obs['pbmc_sample_id'].unique():
        barcodes=adata.obs[adata.obs['pbmc_sample_id'] == i].index.tolist()
        predictions.predicted_labels.loc[barcodes,:].to_csv("Labels_Celltypist/" + i + "_High.csv")
    
    predictions = celltypist.annotate(adata, model='Healthy_COVID19_PBMC.pkl', majority_voting=True)
    for i in adata.obs['pbmc_sample_id'].unique():
        barcodes=adata.obs[adata.obs['pbmc_sample_id'] == i].index.tolist()
        predictions.predicted_labels.loc[barcodes,:].to_csv("Labels_Celltypist/" + i + "_Covid_Healthy.csv")
    del adata
if __name__ == '__main__':
    # Divide the meta_data into subsets
    meta_data_subsets = [meta_data["file.path"][i:i+10] for i in range(0, len(meta_data), 10)]

    # Process each subset in parallel
    with concurrent.futures.ProcessPoolExecutor(max_workers=11) as executor:
        futures = [executor.submit(process_data, meta_data_sub) for meta_data_sub in meta_data_subsets]
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f'Error: {e}')

🔬 Input data has 156449 cells and 33538 genes
🔗 Matching reference genes in the model
🔬 Input data has 193957 cells and 33538 genes
🔗 Matching reference genes in the model
🧬 5967 features used for prediction
⚖️ Scaling input data
🔬 Input data has 187700 cells and 33538 genes
🔗 Matching reference genes in the model
🔬 Input data has 179276 cells and 33538 genes
🔗 Matching reference genes in the model
🔬 Input data has 173666 cells and 33538 genes
🔗 Matching reference genes in the model
🔬 Input data has 207788 cells and 33538 genes
🔗 Matching reference genes in the model
🔬 Input data has 199483 cells and 33538 genes
🔗 Matching reference genes in the model
🔬 Input data has 200104 cells and 33538 genes
🔗 Matching reference genes in the model
🔬 Input data has 198214 cells and 33538 genes
🔗 Matching reference genes in the model
🔬 Input data has 187235 cells and 33538 genes
🔗 Matching reference genes in the model
🧬 5967 features used for prediction
🔬 Input data has 209206 cells and 33538 genes
